# DATA PREPARATION IN BREACHES.

In [200]:
import chardet
with open("prcbreaches2005-18.csv", 'rb') as rawdata:
    result = chardet.detect(rawdata.read(10000))
print(result)

{'encoding': 'Windows-1252', 'confidence': 0.7019810807872132, 'language': ''}


In [201]:
import csv
import pandas as pd
import json

index=pd.DataFrame(columns=['Holi'])
with open('prcbreaches2005-18.csv', 'r', encoding='Windows-1252', errors='ignore') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',')
    cols=pd.DataFrame(columns=['Holi'])
    x=0
    # Defining the columns in my DataFrame
    for j in spamreader:
        if x==0:
            cols=j
            index=pd.DataFrame(columns=cols)
            x+=1
        else:
            index=index.append(pd.DataFrame([j], columns=cols))
            
index.head()

,Records.Breached,Records.Breached...Detail,Name,CatID,Total.Records,Region,Contact..etc.,Category_dm,Entity_prc,State,OrgID,Location,Date.Made.Public,Year,Source.of.Breach.Notification,Type,Description
0,NA,NA,Dean Health Plan,28,"1,311",NA,NA,Discharge Data,MED,Wisconsin,200003106,,15-Jun-18,2018,US Department of Health and Human Services,DISC,Location of breached information: Paper/Films\...
0,NA,NA,"HealthEquity, Inc.",28,"16,000",NA,NA,Discharge Data,MED,Utah,200003107,,12-Jun-18,2018,US Department of Health and Human Services,HACK,Location of breached information: Email\nBusin...
0,NA,NA,Ticketfly,15,"27,000,000",NA,NA,Employer,BSR,California,200003097,San Francisco,12-Jun-18,2018,Media,HACK,Ticketfly was the target of a malicious cyber ...
0,NA,NA,PageUp,17,"2,000,000",NA,NA,Payer (Insurer),BSO,,200003099,,12-Jun-18,2018,Media,HACK,"TechRadar reports:PageUp, an Australia-based s..."
0,NA,NA,"Facebook, inc.",17,"3,000,000",NA,NA,Payer (Insurer),BSR,California,200003102,San Francisco,12-Jun-18,2018,Media,DISC,New Scientist reports:DataÎŒŠ—¾from millions o...


# Solving the heterogeneous date format in our data.

In [202]:
from datetime import datetime
fechameses=pd.DataFrame()
fechamesesydia=pd.DataFrame()
index['Date.Made.Public'] = pd.to_datetime(index['Date.Made.Public'])
for i in index['Date.Made.Public']:
    daymonth=i.strftime("%m-%d")
    fechamesesydia=fechamesesydia.append(pd.DataFrame([daymonth], columns=['monthandday_date']))
    month=i.strftime("%m")
    fechameses=fechameses.append(pd.DataFrame([month], columns=['month_date']))
    
index['monthandday_date']=fechamesesydia['monthandday_date']
index['month_date']=fechameses['month_date']
index.head()

# Let's convert to numeric the column Records.breached, as it is now string.
index['Records.Breached']=index['Records.Breached'].replace('NA','')
index['Records.Breached']=pd.to_numeric(index['Records.Breached'])



# Day of the week visualization

 As it is could be understandable, the dates breaches are made public are consistently less common in non-working days. Thus, in order to represent the frequency, we might be interested in excluding them for some time analysis.


In [203]:
weekdays=pd.DataFrame()
for i in index['Date.Made.Public']:
    dayoftheweek=i.strftime("%A")
    weekdays=weekdays.append(pd.DataFrame([dayoftheweek], columns=['numberofday']))

weekdays=weekdays.reset_index().groupby('numberofday').count()
    

# Sorting the values to begin on Sunday, and visualize it correctly.
letssort = ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']
sortIndex = dict(zip(letssort,range(len(letssort))))
sortIndex
weekdays['nameofday'] = weekdays.index.map(sortIndex)  
weekdays.sort_values('numberofday', inplace=True)

weekdays.to_csv('weekdays.csv')

In [204]:
daymonth_onlyworkdays=pd.DataFrame()
for i in index['Date.Made.Public']:
    if int(i.strftime("%u")) <=5:
        onlyworkdays=i.strftime("%m-%d")
        daymonth_onlyworkdays=daymonth_onlyworkdays.append(pd.DataFrame([onlyworkdays], columns=['onlyworkdays']))
fechasmeses=daymonth_onlyworkdays.reset_index().groupby('onlyworkdays').count()


weekdays.head()


,index,nameofday
numberofday,,
Friday,1115,5
Monday,667,1
Saturday,157,6
Sunday,74,0
Thursday,775,4


In [205]:
fechasmeses=index.groupby('month_date').count()
fechasmesesydias=index.groupby('monthandday_date').count()


fechasmeses=pd.concat([fechasmeses['monthandday_date'], fechasmeses['Name']], axis=1)
fechasmesesydias=pd.concat([fechasmesesydias['month_date'], fechasmesesydias['Name']], axis=1)

In [206]:
index[1:].to_csv('pruebasbreaches.csv')
fechasmeses.to_csv('pruebasbreaches_fechasmeses.csv')
fechasmesesydias.to_csv('pruebasbreaches_fechasmesesydias.csv')


# Node graphs visualization

In [207]:
#First let's convet CatID to integers.
index['CatID']=pd.to_numeric(index['CatID'])
index.sort_values('CatID', inplace=True)
# Did we do it correctly?
index.head()

# Let's count number of breaches reported by CatID.
countbreachesbycatID=index.groupby('CatID').count()
countbreachesbycatID=countbreachesbycatID.filter(['Name'])

# Let's also count records breached by CatID:
countrecordsbycatID=index.groupby('CatID').sum()
countrecordsbycatID=countrecordsbycatID.filter(['Records.Breached'])
countcatids=pd.merge(countrecordsbycatID, countbreachesbycatID, right_index=True, left_index=True)

# Some CatIDs are not represented in our database, so let's merge it with the information dataset we've been provided.
categories = pd.read_csv('C:/Users/Fran/Trabajo-ENS/Harvard TheDataMap/Nueva carpeta/categories.csv', sep=',', encoding='Windows-1252')
mergecatids=pd.merge(categories, countcatids, left_on='CatID', right_index=True, how='outer')
# Let's replace by 0 the values of the CatIDs which did not report any breach, or whose breaches did not report records breached.
mergecatids['Name'].fillna(value=0.0, inplace=True)
mergecatids['Records.Breached'].fillna(value=0.0, inplace=True)
mergecatids.to_json('nodes.json', orient='records')

edges = pd.read_excel(io='C:/Users/Fran/Trabajo-ENS/Harvard TheDataMap/Nueva carpeta/edges.xlsx')
edges.columns=(['source', 'target', 'noname'])
edges.to_json('edges.json', orient='records')


with open('edges.json', 'r') as f:
    edgesjson=json.load(f)
with open('nodes.json', 'r') as f:
    nodesjson=json.load(f)
datajson={'nodes': nodesjson, 'edges': edgesjson}
with open('nodegraphdata.json', 'w') as outfile:
    json.dump(datajson, outfile)
  

# US States map visualization

In [214]:
#Let's first summarize number of breaches by state.
countrecordsperstate=index.groupby('State').count()
countrecordsperstate2=countrecordsperstate['Name']
print(countrecordsperstate2)

# Although we won't use the sum of records in the US states viz, the info could be useful for further visualizations.
sumrecordsperstate=index.groupby('State').sum()
sumrecordsperstate2=sumrecordsperstate['Records.Breached']
print(sumrecordsperstate2)

holi=pd.concat([sumrecordsperstate2, countrecordsperstate2],  axis=1)
print(holi.columns)
holi.columns=['numberofrecords', 'numberofbreaches']
holi.to_csv('statescount.csv')


# Now let's merge with the spatial shapefile we already have. 
import os
import re

with open('C:/Users/Fran/Trabajo-ENS/Harvard TheDataMap/Nueva carpeta/NuevoenGitHub/us-states.js') as dataFile:
    data = dataFile.read()
    # I want to only extract the variable information. In the end, we'll put it back.
    obj = data[data.find('{'):data.rfind('}')+1]
    jsonObj = json.loads(obj)
    
with open('statescount.csv', 'r') as csvfile_states:
    spamreader = csv.reader(csvfile_states, delimiter=',')
    x=0
    # Defining the columns in my DataFrame
    for j in spamreader:
        if x==0:
            cols=j
            index=pd.DataFrame(columns=cols)
            x+=1
        else:
            index=index.append(pd.DataFrame([j], columns=cols))
            
# now that it's already been built, let's jump to merge both.
index=index.replace('District Of Columbia', 'District of Columbia')
j=0
for h in index['State']:
    probemos=index['numberofbreaches'].iloc[j]
    j+=1
    for i in jsonObj['features']:
        if i['properties']['name']==h:
            print("Worked with " + i['properties']['name'])
            i['properties']={'name': h, 'density': i['properties']['density'], 'numberofbreaches': int(probemos)}
            
f = open("C:/Users/Fran/Trabajo-ENS/Harvard TheDataMap/Nueva carpeta/NuevoenGitHub/us-states2.js", 'w')
f.write("var statesData = " + str(jsonObj))
f.close()

State
                         30
Alabama                  45
Alaska                   12
Argentina                 1
Arizona                  69
Arkansas                 31
Beijing                   1
Berlin                    1
British Columbia          3
Buckinghamshire           2
California              756
Cheshire                  1
Colorado                 76
Connecticut              58
Delaware                  9
District Of Columbia     53
Dublin                    1
Florida                 212
Georgia                 111
Grand Bahama              1
Guangdong                 1
Hawaii                   10
Idaho                    10
Illinois                153
Indiana                  96
Iowa                     35
Kansas                   31
Kentucky                 62
London                    1
Louisiana                28
                       ... 
Missouri                 70
Montana                  14
Nebraska                 12
Nevada                   30
New Hampshire 